In [1]:
import pandas as pd
from IPython.display import display
from tqdm import tqdm

from openpyxl import Workbook
from openpyxl.styles import Alignment, Font
from openpyxl.utils.dataframe import dataframe_to_rows

In [2]:
pd.options.display.float_format = '{:,.2f}%'.format

In [3]:
data_pkl = './datasus_tidy.pkl'
xl_file = './summary.xlsx'
missing_data = 'S.I.'

In [4]:
data = pd.read_pickle(data_pkl, compression='zip')

In [5]:
cols = ['EstadoCivil', 'Sexo', 'RacaCor', 'Escolaridade', 'LocalOcorrenciaObito']
lbls = ['Estado Civil', 'Sexo', 'Raça/Cor', 'Escolaridade', 'Local de Ocorrência do Óbito']

In [6]:
wb = Workbook()
ws = wb.active

ws.column_dimensions['A'].width = max([len(lbl) for lbl in lbls])
ws['A1'] = 'Características de vítimas fatais de acidentes de trânsito (2006-2018)'
ws['A1'].alignment = Alignment(horizontal='left', vertical='top')
ws['A1'].font = Font(sz=14, bold=True)
ws.append([])

ncols = data.DataObito.dt.year.max() - data.DataObito.dt.year.min() + 1
ws.merge_cells(start_row=1, end_row=ws.max_row+1, start_column=1, end_column=ncols+1)

for col, lbl in tqdm(zip(cols, lbls)):

    if data[col].dtype == 'category':
        data[col].cat.add_categories(missing_data, inplace=True)
        data.loc[data[col].isna(), col] = missing_data

    pvt = pd.pivot_table(data, index=data[col], columns=data.DataObito.dt.year, values='DataObito', aggfunc='count')
    pvt = (pvt/pvt.sum())
    pvt = pvt[pvt.index != missing_data].sort_values(by=pvt.columns[0], ascending=False)\
                                        .append(pvt[pvt.index == missing_data])
    pvt.index.name = lbl
    
    for row in dataframe_to_rows(pvt, header=(col==cols[0])):
        ws.append(row)
        
        # Row with years
        if row[0] is None:
            for cell in ws[ws.max_row][1:]:
                cell.font = Font(bold=True)

        # Rows with variables' names
        if row[0] in lbls:
            ws[ws.max_row][0].alignment = Alignment(indent=5)
            ws[ws.max_row][0].font = Font(bold=True)
            ws[ws.max_row][0].value = ws[ws.max_row][0].value.upper()
            ws.merge_cells(start_row=ws.max_row, end_row=ws.max_row, start_column=1, end_column=ncols+1)
            
        # Formatting percentages
        for cell in ws[ws.max_row]:
            if type(cell.value) == float:
                cell.number_format = '0.00%'

wb.save(xl_file)

5it [00:00,  9.91it/s]
